In [138]:
#| default_exp appmod_with_genai

# appmod_with_genai -- Accelerate your AppMod journey with GenAI.

> Hello Bedrock help me accelerate my Application Modernisation. 

In [135]:
#| hide
from nbdev.showdoc import *

In [136]:
#| export
def foo(): pass

In [137]:
#| hide 
import nbdev; nbdev.nbdev_export()

 # Introduction
> Add later.

In [129]:
import json
import os
import sys

import boto3
import asyncio
from typing import List

from langchain_community.llms import Bedrock
from langchain_community.chat_models import BedrockChat
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_community.document_loaders import GithubFileLoader


model_id = "mistral.mistral-large-2402-v1:0"


bedrock_runtime = boto3.client('bedrock-runtime')

llm = BedrockChat(
    client=bedrock_runtime,
    model_id=model_id,
)

chat = BedrockChat(model_id=model_id)

In [121]:
! pip install asyncio
# parser = StrOutputParser()

In [122]:
# If you haven't set your access token as an environment variable, pass it in here.
from getpass import getpass

ACCESS_TOKEN = getpass()


 ········


In [124]:
loader = GithubFileLoader(
    repo="jihed/helloworld-mdb",
    access_token=ACCESS_TOKEN,  # delete/comment out this argument if you've set the access token as an env var.
    branch= "main",
    github_api_url="https://api.github.com",
    file_filter=lambda file_path: file_path.endswith(
        "HelloWorldTopicMDB.java"
    ),  # load all java files.
)


In [125]:
sample_source_code= loader.load()

In [130]:
print(sample_source_code)

[Document(page_content='/*\n * JBoss, Home of Professional Open Source\n * Copyright 2015, Red Hat, Inc. and/or its affiliates, and individual\n * contributors by the @authors tag. See the copyright.txt in the\n * distribution for a full listing of individual contributors.\n *\n * Licensed under the Apache License, Version 2.0 (the "License");\n * you may not use this file except in compliance with the License.\n * You may obtain a copy of the License at\n * http://www.apache.org/licenses/LICENSE-2.0\n * Unless required by applicable law or agreed to in writing, software\n * distributed under the License is distributed on an "AS IS" BASIS,\n * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n * See the License for the specific language governing permissions and\n * limitations under the License.\n */\npackage org.jboss.as.quickstarts.mdb;\n\nimport java.util.logging.Logger;\nimport javax.ejb.ActivationConfigProperty;\nimport javax.ejb.MessageDriven;\nimport jav

In [ ]:
!pip install -qU langchain-text-splitters

In [ ]:
# Full list of supported languages
[e.value for e in Language]

In [ ]:
RecursiveCharacterTextSplitter.get_separators_for_language(Language.JAVA)

# Stuff Documents

This is an advanced prompt, where we ask the LLM:
1 - ask to generate migration plan with code snippet on markdown file.

> Todo: Add pom.xml to the model to provide request.

In [112]:
from langchain_core.documents import Document
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_messages(
    [("system", "Generate for me a step by step guide in mardown format to migrate all the documents in context written in enterprise Java EE source code to Quarkus:\n\n{context}")]
)
chain = create_stuff_documents_chain(llm, prompt)
#print(print(documents))

response = chain.invoke({"context": sample_source_code})

print(response, end= '\n')



Sure, here's a step-by-step guide in Markdown format to help you migrate your Java EE application to Quarkus. Please note that this is a general guide and the exact steps might vary depending on the specifics of your application.

```markdown
# Migrating Java EE Applications to Quarkus

This guide provides a step-by-step process to migrate a Java EE application to Quarkus.

## Prerequisites

- Basic understanding of Java EE and Quarkus
- Java Development Kit (JDK) installed on your system
- Apache Maven or Gradle for building your application

## Step 1: Analyze Your Application

Before starting the migration process, analyze your application to understand its architecture, dependencies, and functionalities. This will help you identify the components that need to be migrated and the potential challenges you might face.

## Step 2: Set Up Quarkus

Set up a new Quarkus project using the Quarkus Maven or Gradle plugin. You can also use the Quarkus code generator to create your project.


# Refine Document
This is an advanced prompt, where we ask the LLM:
1.Build a Migration Plan from Java EE to Quarkus
2.Based on that plan, we ask to generate Quarkus code.

> Todo: Add pom.xml to the model to provide request.

In [132]:
from langchain.chains import RefineDocumentsChain, LLMChain
from langchain_core.prompts import PromptTemplate

# This controls how each document will be formatted. Specifically,
# it will be passed to `format_document` - see that function for more
# details.
document_prompt = PromptTemplate(
    input_variables=["page_content"],
     template="{page_content}"
)
document_variable_name = "context"

# The prompt here should take as an input variable the
# `document_variable_name`
prompt = PromptTemplate.from_template(
    "[INST] Build a migration plan from Java Entreprise EE to Quarkus for this content: {context} [/INST]"
)
initial_llm_chain = LLMChain(llm=llm, prompt=prompt)
initial_response_name = "prev_response"
# The prompt here should take as an input variable the
# `document_variable_name` as well as `initial_response_name`
prompt_refine = PromptTemplate.from_template(
    "[INST] Here's your Migration Plan: {prev_response}. Now generate a quarkus code for: {context} [/INST]"
)
refine_llm_chain = LLMChain(llm=llm, prompt=prompt_refine)
chain = RefineDocumentsChain(
    initial_llm_chain=initial_llm_chain,
    refine_llm_chain=refine_llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
    initial_response_name=initial_response_name,
)



response = chain.invoke(sample_source_code)



In [131]:
#print(response.get('output_text'), end= '\n')
type(response.get('output_text'))

with open("output.txt", "w") as f:
    f.write(response.get('output_text'))